In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import surp
import matplotlib.pyplot as plt
import vice
import numpy as np
from surp.gce_math import Z_to_MH, MH_to_Z, Z_SUN
from surp.yields import calc_y
import arya

from surp.agb_interpolator import interpolator


In [ ]:
y_cc = vice.yields.ccsne.settings
y_agb = vice.yields.agb.settings

In [ ]:
def plot_y_z(Zs=Z_SUN * np.logspace(-1, 0.5, 100), ele="c", yields="all", kind="line", factor = 1, **kwargs):        
    y_c_agb = calc_y(Zs, ele=ele, kind=yields)
    plt.xlabel(r"$\log Z / Z_\odot$")
    plt.ylabel(r"yield")
    if kind == "line":
        p = plt.plot(Z_to_MH(Zs), y_c_agb, **kwargs)
    elif kind == "scatter":
        p = plt.scatter(Z_to_MH(Zs), y_c_agb, **kwargs)
    return p


# CCSNe Model

In [ ]:
from surp.yield_models import Lin_CC, LogLin_CC, BiLin_CC, BiLogLin_CC, C_AGB_Model, ZeroAGB, Quadratic_CC, Piecewise_CC

In [ ]:
y_mg = y_cc["mg"]
noneq_factor = 1.025
noneq_zeta = 1.8
noneq_A = 1.8

Y_C_0 = 4.12 * y_mg * noneq_factor
ZETA_C_0 = 1.21 * y_mg * noneq_factor * noneq_zeta
A_C_0 = 3.07 * y_mg * noneq_factor * noneq_A

In [ ]:
BiLin_CC(zeta=0.1, y0=2)(2)

In [ ]:
LogLin_CC(zeta=ZETA_C_0, y0=Y_C_0)(1)

In [ ]:
vice.yields.agb.settings["c"] = ZeroAGB()

In [ ]:
y0 = Y_C_0

In [ ]:
y_cc["c"] = Lin_CC(zeta=ZETA_C_0, y0=y0 )
plot_y_z(label="linear", ls="--")

y_cc["c"] = BiLin_CC(zeta=ZETA_C_0, y0=y0, Z1=0.008, y1=0.0001)
plot_y_z(label="BiLin", ls="--")


y_cc["c"] = LogLin_CC(zeta=ZETA_C_0, y0=Y_C_0)
plot_y_z(label="loglin")

y_cc["c"] = BiLogLin_CC(zeta=ZETA_C_0, y0=Y_C_0, y1=0.001)
plot_y_z(label="BiLogLin",  ls=":")


y_cc["c"] = Quadratic_CC(A=A_C_0, zeta=ZETA_C_0,  y0=Y_C_0)
plot_y_z(label="quadratic")


arya.Legend(-1)

In [ ]:
y_cc["c"] = Quadratic_CC(A=0, zeta=ZETA_C_0,  y0=Y_C_0)
plot_y_z(label="quadratic")

In [ ]:
y0s = [4e-4, 2e-4, 2.5e-4]
Zs = [1e-4, 0.008]
zetas = [0, 1e-4, 2.5e-4]
y_cc["c"] = Piecewise_CC(y0s, zetas, Zs)

MH = np.linspace(-5, 0.5, 1000)
Z = MH_to_Z(MH)
plot_y_z(Zs=Z, label="quadratic")




In [ ]:
y_cc["c"] = LogLin_CC(zeta=0.001)

plot_y_z()

y_cc["c"] = LogLin_CC(y0 = 0.005, zeta=0.001)
plot_y_z()

y_cc["c"] = lambda Z: LogLin_CC(y0 = 0.005, zeta=0.001)(Z) - LogLin_CC(y0 = 0.004, zeta=0.001)(Z)
plot_y_z(label="difference")
arya.Legend(loc=-1)

In [ ]:
y_cc["c"] = LogLin_CC(y0=0, zeta=0.01)

plot_y_z()

y_cc["c"] = LogLin_CC(y0=0, zeta = 0.02)
plot_y_z()

arya.Legend(loc=-1)

In [ ]:
plot_y_z()

In [ ]:
y_cc["c"] = Lin_CC()
Zs = MH_to_Z(np.linspace(-4, 1, 100))
plot_y_z(Zs)

y_cc["c"] = BiLin_CC(Z1=0.001, y1=2e-3)

plot_y_z(Zs,)
arya.Legend(loc=-1)
plt.axhline(2e-3)
plt.axvline(Z_to_MH(1e-3))
plt.ylim(0, 3e-3)

# AGB Interpolator

In [ ]:
y_cc["c"] = 0
y_agb = vice.yields.agb.settings

In [ ]:
for study in surp.AGB_MODELS:
    
    y_agb["c"] = interpolator("c", study=study, interp_kind="log", low_z_flat=True)
    plot_y_z( label=study)

arya.Legend(loc=-1)

In [ ]:
for alpha in [0.5, 1, 2]:
    y_agb["c"] = interpolator("c", prefactor=alpha)
    plot_y_z( label=alpha)
    y_agb["c"] = interpolator("c")
    y_agb["c"] = y_agb["c"]*alpha
    plot_y_z(ls="--")

arya.Legend(loc=-1)

In [ ]:
ms = np.linspace(0, 10, 1000)
Z = 0.1*Z_SUN
for alpha in [0.5, 1, 2]:
    y_agb["c"] = interpolator("c", mass_factor=alpha, pinch_mass=1)
    plt.plot(ms, [y_agb["c"](m, Z) for m in ms], label=alpha)

arya.Legend(loc=-1)
plt.xlabel("mass")
plt.ylabel("Y")

In [ ]:
for alpha in [0.5, 1, 2]:
    y_agb["c"] = interpolator("c", mass_factor=alpha, no_negative=True)
    plt.plot(ms, [y_agb["c"](m, Z) for m in ms], label=alpha)

arya.Legend(loc=-1)
plt.xlabel("mass")
plt.ylabel("Y")

In [ ]:
interpolator("c")(7.999999999999999, 0.014)

In [ ]:
8 * 1.3

In [ ]:
print(_ / 1.3)

In [ ]:
Zs = 0.016*10**np.linspace(-3, 0.5)
y_agb["c"] = interpolator("c")
plot_y_z(Zs)

y_agb["c"] = interpolator("c", interp_kind="log")
plot_y_z(Zs)

y_agb["c"] = interpolator("c", interp_kind="log", low_z_flat=True)
plot_y_z(Zs)


y_agb["c"] = interpolator("c", low_z_flat=True)
plot_y_z(Zs)